# Spamku 
## April 4, 2020
## Version 2.0.1

### Import Libraries and Setup Variables

Import nltk libraries

Setup dictionaries for storing learning

In [87]:
# https://www.nltk.org/
import nltk;

# http://www.nltk.org/book/ch05.html
from nltk import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('cmudict')
from nltk.corpus import cmudict

# https://docs.python.org/3/library/os.html
import os

# https://docs.python.org/3/library/pickle.html
import pickle

# https://docs.python.org/3/library/collections.html
from collections import defaultdict


pos_dict=defaultdict(list)
# declare custom dictionary for determining part of speech
pro = cmudict.dict()

# https://docs.python.org/3/library/random.html
import random


import string



[nltk_data] Downloading package punkt to /home/parallels/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/parallels/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to
[nltk_data]     /home/parallels/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


### Read in Corpus of Sample Spam

This file contains 8893 spam-ku


In [5]:
filename="corpus/spamku.txt"
file = open(filename, mode = 'r')
#text=file.read()
text = ""

for line in file:
    text += line.strip()+' '
file.close()

### Perform Markov Chain Analysis on Sample Poems

Determining the most likely word to follow each word in sample

In [108]:
def markov_chain(text):
    '''The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text.'''
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = defaultdict(list)
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)

    # Convert the default dict back into a dictionary
    m_dict = dict(m_dict)
    return m_dict

mc=markov_chain(text)

def hormel_chain(text):
    '''The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text.'''
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = [] #defaultdict(list)
    invalid = 0
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        try:
            # print(current_word)
            current_word = current_word.lower().translate(str.maketrans('','',string.punctuation))
            count = get_syllable_count(current_word)
            pos = get_pos_label([current_word])
            m_dict[count][pos].append(next_word)
        except:
            invalid += 1
    # Convert the default dict back into a dictionary
    # m_dict = dict(m_dict)
    return m_dict

hc=hormel_chain(text)

print(hc)

[]


### Define functions to count syllables for each word

Testing with 3 words

In [78]:
def get_syllable_count(word):
    # print(pro[word])
    syl=[ x for x in pro[word][0] if x[-1].isdigit()]
    return len(syl)

print(get_syllable_count('pastrami'))
print(get_syllable_count('sausage'))
print(get_syllable_count('ribs'))


3
2
1


### Define function to return part of speech for each word

Testing with 3 words

In [28]:
def get_pos_label(word):
    return pos_tag(word)[0][1]

print(get_pos_label(['capicola']))
print(get_pos_label(['pastrami']))
print(get_pos_label(['andouille']))


NN
NN
NN


### Process each line of sample

Determine the arrangement of syllables

Determine the parts of speech of each word

### Pick a pattern for each line

Choose a syllable sequence for the 1st, 2nd and 3rd line

### Seed the poems 1st word of syllable length from pattern

In [51]:
chain=markov_chain(text)

# Capitalize the first word
word1 = random.choice(list(chain.keys()))
# https://www.programiz.com/python-programming/methods/string/capitalize
sentence = word1.capitalize()


print(sentence)

Slots


### Use chain to pick next word with correct number of syllables